<a href="https://colab.research.google.com/github/JacobDowns/CSCI-491-591/blob/main/lecture7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Programming

* Following up on our discussion of parallelism, we'll move onto GPU programming
* A great resource for an introduction to GPU programming can be found [here](https://enccs.github.io/gpu-programming/1-gpu-history/)
* GPUs were originally developed for the highly parallel task of graphics processing, but have since become a workhorse in many applications including AI
* GPUs are specialized parallel hardware for floating point operations
* In traditional systems, the CPU still controls the work flow and designates highly parallel tasks to the GPU







# Why Use GPUs
* **Speed**: GPU computing can offer massive performance benefits for some problems
* **Efficiency**: Compared to CPUs, GPUs can perform more calculations per watt for certain workloads
* **Cost Effectiveness:** GPUs can also be more cost-effective than traditional CPU-based systems for some workloads
* **As an Excuse:** You need a high end GPU for "work", but you might as well test it out on some video games


# GPU Hardware
* CPUs have a moderate number of complex and powerful cores
* GPUs have a multitude of simpler cores designed for highly parallel, high-throughput problems
* GPU cores are not as powerful and they are not designed for general purpose computing
* However, GPUs have extremely high core counts. For example, an RTX 5090 has **21,760** CUDA cores + other cores for ray tracing and tensor processing
* CPUs can efficiently handle **task parallelism**, which involves running several diverse tasks concurrently
* GPUs are good at **data parallelism** in which the same operation is applied in parallel to different chunks of the data
* In contrast to CPUs, GPUs contain a relatively small amounts of transistors dedicated to control and caching, and a much larger fraction of transistors dedicated to the mathematical operations

<figure>
<img src="https://enccs.github.io/gpu-programming/_images/CPUAndGPU.png" width="1000px">
<figcaption align="center"> CPU v. GPU Architecture from ENCCS
</figure>

<table border="1" style="border-collapse: collapse; text-align: center; font-size:18px;">
  <tr>
    <th style="padding:8px;">CPU</th>
    <th style="padding:8px;">GPU</th>
  </tr>
  <tr>
    <td style="padding:8px;">General purpose</td>
    <td style="padding:8px;">Highly specialized for parallelism</td>
  </tr>
  <tr>
    <td style="padding:8px;">Good for serial processing</td>
    <td style="padding:8px;">Good for parallel processing</td>
  </tr>
  <tr>
    <td style="padding:8px;">Great for task parallelism</td>
    <td style="padding:8px;">Great for data parallelism</td>
  </tr>
  <tr>
    <td style="padding:8px;">Low latency per thread</td>
    <td style="padding:8px;">High-throughput</td>
  </tr>
</table>

## Problems Suited to GPUs
* As a general rule of thumb:

> **Good on GPU:**
  * same operation on lots of data
  * emmbarassingly parallel
  * compute-heavy

**Examples:**

* Many large-scale matrix and vector operations
  * Matrix multiplication
  * Element-wise operations on arrays where the same operation is performed on every element
  * Factorization and iterative linear solvers work reasonably well on GPUs
  * These linear algebra operations are often found in machine learning, scientific computing, and image processing applications
* Fourier Transforms
  * Fourier transforms can be parllelized efficienctly
  * They are also used in machine learning, scientific computing, data science, and image processing
* Monte Carlo Simulations
  * Used across finance, physics, and other fields to simulate complex systems
* Deep learning workloads



## Problems Not Suited to GPUs
* As a general rule of thumb:

> **Bad on GPU:**
 * lots different operations
 * little work per element
 * sequential problems
 *  memory/branch dominated problems


 ### Memory and Branch Dominated Problems

* GPUs are not especially good at **memory-bound** problems that are limited by how fast you can move data in and out of memory instead of doing computation
* They are also not good at **branch-dominated** problems where performance is limited by control flow like lots of if/else decisions, irregular loops, or recursion

**Examples:**
* Sparse Linear Algebra
  * Sparse matrix-vector multiply is memory-bound
* Graph algorithms
  * Things like breadth-first search depth-first search with irregular memory access
* Control-heavy Code
  * Code with lots of branching logic, dynamic programming, or irregular loops
* Small Problems
  * Problems need to be big enough to justify copying data to the GPU memory and launching compute kernels
* Sequential Tasks



## CUDA Basics
* Although we will not be programming CUDA kernels directly in C or C++, we will look at some CUDA code to illustrate a few lower level concepts that will be useful when doing GPU programming in Python
* In order to perform operations on the GPU we need to create a function called a **kernel** that will be executed simultaneously across many threads running on the GPU in parallel
* For example, we could have a basic kernel that takes in pointers to some integer arrays and computes their element-wise sum

```c
__global__ void addKernel(int *c, const int *a, const int *b, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        c[i] = a[i] + b[i];
    }
}
```

* We might use this kernel in the context of the following program where we create some arrays of integers, copy them to GPU memory, launch the kernel, then copy the results back to the host.

```c
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void addKernel(int *c, const int *a, const int *b, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    const int size = 5;
    int a[size] = {1, 2, 3, 4, 5};
    int b[size] = {10, 20, 30, 40, 50};
    int c[size] = {0};

    // Allocate GPU memory
    int *dev_a, *dev_b, *dev_c;
    cudaMalloc(&dev_a, size * sizeof(int));
    cudaMalloc(&dev_b, size * sizeof(int));
    cudaMalloc(&dev_c, size * sizeof(int));

    // Copy inputs to GPU
    cudaMemcpy(dev_a, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel
    addKernel<<<1, size>>>(dev_c, dev_a, dev_b, size);

    // Copy result back to host
    cudaMemcpy(c, dev_c, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    for(int i = 0; i < size; i++) {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    // Free GPU memory
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


* Most of this looks like pretty standard C code, but there are a few unique pieces
* For instance what  are `blockIdx`, `blockDim`, and `threadIdx`?
* When launching the kernel, what exactly does `addKernel<<<1, size>>>` mean?
* To understand these lines, we need to learn about CUDA threads


### CUDA Threads

* Kernels are executed on many CUDA threads simultaneously
* Every thread is associated with a particular intrinsic index which can be used to calculate and access particular memory location in an array
* Each thread has its context and set of private variables
* Threads are organized into a hierarchy:
  * Threads are grouped into groups of 32 threads called **warps**
  * All threads in a warp execute the same instruction at the same timee (SIMT: Single Instruction, Multiple Threads).
  * Multiple warps are organized into **blocks**
  * The hardware schedulaer assigns a block to an SMP or (Streaming Mulitiprocessor Unit) which has its own local, memory
  * Blocks are organized into a **grid**
* This organizational structure is related both to the physical architecture of the GPU as well as being a software abstraction
* This hierarchy also controls memory access:
  * Global memory is accessible by all threads
  * Shared memory is accessible between threads in a block
  * Private memory is private to each thread
  * Constant memory is read-only memory accessible to all threds


* In the example kernel, the block index, thread index, and block dimension are used to index into the arrays `a`, `b`, and `c`
* That is, these indexes identify what thread is working and what subset of the data to operate on
* The image below shows an example assuming a block size of 256
<figure>
<img src="https://enccs.github.io/gpu-programming/_images/Indexing.png" width="1000px">
<figcaption align="center"> How threads in a grid can be associated with specific elements of an array from ENCCS
</figure>


* When launching a kernel you specify both the number of blocks and the number of threads
* For example this would launch 80 blocks with 256 threads each
```c
myKernel<<<80, 256>>>(...);
```
* The number of threads per block is usually limited to 1024. On older GPUs this was 512.
* You can only synchronize threads within a block. Blocks run independently and the GPU can schedule them flexibly


### Shared Memory Example
* In the above example of the add kernel, we only had one block with five threads
* Each thread was responsible for doing a single summation
* Values were fetched from global memory
* This was perfectly fine, but we could also use shared memory in cases where data is frequently accessed  
```python
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void add_shared(int *c, const int *a, const int *b, int n) {
    extern __shared__ int s[];          // dynamic shared buffer
    int* s_a = s;                       // first half for a
    int* s_b = s + blockDim.x;          // second half for b

    int tx = threadIdx.x;
    int i  = blockIdx.x * blockDim.x + tx;

    // Load GLOBAL -> SHARED (bounds check for last, partial block)
    if (i < n) {
        s_a[tx] = a[i];
        s_b[tx] = b[i];
    }
    __syncthreads();                    // ensure all loads completed

    // Compute from SHARED, write to GLOBAL
    if (i < n) {
        c[i] = s_a[tx] + s_b[tx];
    }
}

int main() {
    // Example data (same size for a, b, c)
    const int n = 5;
    int h_a[n] = {1, 2, 3, 4, 5};
    int h_b[n] = {10, 20, 30, 40, 50};
    int h_c[n] = {0};

    // Allocate device (GPU) memory
    int *d_a = nullptr, *d_b = nullptr, *d_c = nullptr;
    cudaMalloc(&d_a, n * sizeof(int));
    cudaMalloc(&d_b, n * sizeof(int));
    cudaMalloc(&d_c, n * sizeof(int));

    // Copy inputs host -> device
    cudaMemcpy(d_a, h_a, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(int), cudaMemcpyHostToDevice);

    // Launch configuration
    int threadsPerBlock = 256;         
    int numBlocks = (n + threadsPerBlock - 1) / threadsPerBlock;
    size_t shmemBytes = 2 * threadsPerBlock * sizeof(int);      // s_a + s_b

    // Launch shared-memory kernel
    add_shared<<<numBlocks, threadsPerBlock, shmemBytes>>>(d_c, d_a, d_b, n);
    cudaDeviceSynchronize();  

    // Copy result device -> host
    cudaMemcpy(h_c, d_c, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    for (int i = 0; i < n; ++i) {
        printf("%d + %d = %d\n", h_a[i], h_b[i], h_c[i]);
    }

    // Cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}
```

* The main difference in this example is that we specifically create a memory buffer `s` that can be accessed by all threads
* This could be useful if we need to repeatedly access the data (although that isn't the case in this example)



### GPU Programming in Python
* Fortunately, we don't necessarily need to write C or C++ code to use CUDA
* There are a few Python packages for writing CUDA code that can take advantage of GPUs
* We'll discuss two Python packages in particular

#### CuPy — Essentially NumPy on the GPU
* What it is: A NumPy-compatible array library whose ops run on the GPU.
* How you use it: Replace NumPy as np with cupy as cp
* Most NumPy code just works
* Under the hood: Calls CUDA libraries (cuBLAS, cuSOLVER, cuFFT, cuSPARSE, cuRAND, etc.).
* Capabilities
  * Vectorized math, broadcasting, indexing, ufuncs
  * Linear algebra (cp.linalg → cuBLAS/cuSOLVER), FFT (cp.fft → cuFFT), random (cp.random → cuRAND), sparse (cupyx.scipy.sparse → cuSPARSE).
  * Custom kernels without leaving Python

#### Numba - Writing Kernels in Python
* What it is: A JIT compiler that turns a subset of Python into CUDA kernels
* How you use it: Decorate functions with @cuda.jit (or create CUDA ufuncs with @vectorize(target="cuda")).
* Capabilities
  * Full thread/block/grid control; launch with [blocks, threads].
  * Shared memory, atomics, syncthreads(), warp intrinsics (e.g., shuffles), streams/events.
  * Memory management: cuda.to_device, device_array, pinned and managed memory.
* Best for writing custom kernels when no library op fits
* Offers more fine-grained control over blocks/threads/shared memory—all in Python
